### 1. 필요한 라이브러리 임포트

In [12]:
from wca_utils import *
import os
import ast

### 2. wca_utils.py를 사용하여 WCA API 호출 함수 정의

In [2]:
# Define WCA API call function
def call_wca_api(prompt):

    payload = build_basic_prompt_paylod(prompt)
    response = call_wca_url(payload=payload, apikey=iam_apikey, file_dict=[])
    result = (response['response']['message']['content'])

    print("WCA API response: \n ", result)

    return result

### 3. 기존 파이썬 애플리케이션 코드 로드

In [3]:
# Load existing python application code
base_dir = "python-todo-app/"  # Update this path if needed

def load_python_code(file_path):
    """Loads Python source code from a file."""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

python_files = {
    "app.py": load_python_code(base_dir + "app.py"),
    "templates/index.html": load_python_code(base_dir + "/templates/index.html"),
    "tasks.json": load_python_code(base_dir + "tasks.json")
}

python_files

{'app.py': 'from flask import Flask, request, jsonify, render_template\nimport json\nimport os\n\napp = Flask(__name__, template_folder="templates")\n\n# Load tasks from a JSON file\ndef load_tasks():\n    if os.path.exists("tasks.json"):\n        with open("tasks.json", "r") as file:\n            return json.load(file)\n    return []\n\n# Save tasks to JSON file\ndef save_tasks(tasks):\n    with open("tasks.json", "w") as file:\n        json.dump(tasks, file)\n\n# Route to serve the frontend HTML page\n@app.route("/")\ndef home():\n    return render_template("index.html")\n\n# API to get all tasks\n@app.route("/tasks", methods=["GET"])\ndef get_tasks():\n    tasks = load_tasks()\n    return jsonify(tasks)\n\n# API to add a task\n@app.route("/tasks", methods=["POST"])\ndef add_task():\n    tasks = load_tasks()\n    new_task = request.json\n    tasks.append(new_task)\n    save_tasks(tasks)\n    return jsonify({"message": "Task added successfully"}), 201\n\nif __name__ == "__main__":\n  

### 4. 마이그레이션을 위한 프롬프트 정의 및 WCA API 호출

In [11]:
prompt = f"""Here's my sample python application, which take input from user through webpage and stores it in tasks.json. 
        Here's dict structure with keys are filenames and values as the corresponding code of my python application:
        ``` {python_files} ```

        I want to migrate my python application to springboot framework. I'm using Maven 3.9.9 and java 21.0.4. Follow below instructions:
        - Generate the corresponding springboot application code in dict structure, where keys are filepaths and corresponding values contain code. 
        - Make sure, the generated springboot code runs similar to python application with proper webpage and taking data from user and stores the data into tasks.json.
        - Also generate all the necessary files for a successful maven build.

        Now generate the code in dict
        """
response = call_wca_api(prompt)
response

WCA API response: 
  Sure, I can help you with that. Here's the migrated Spring Boot application code in a dictionary, where the keys are filepaths and the values are the code:

```
# Assisted by watsonx Code Assistant 
{
  'pom.xml': '<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">\n  <modelVersion>4.0.0</modelVersion>\n  <groupId>com.example</groupId>\n  <artifactId>demo</artifactId>\n  <version>0.0.1-SNAPSHOT</version>\n  <dependencies>\n    <dependency>\n      <groupId>org.springframework.boot</groupId>\n      <artifactId>spring-boot-starter-web</artifactId>\n    </dependency>\n  </dependencies>\n  <build>\n    <plugins>\n      <plugin>\n        <groupId>org.springframework.boot</groupId>\n        <artifactId>spring-boot-maven-plugin</artifactId>\n      </plugin>\n    </plugins>\n  </build>\n</project>',
  'src/main/java/com/e

'Sure, I can help you with that. Here\'s the migrated Spring Boot application code in a dictionary, where the keys are filepaths and the values are the code:\n\n```\n# Assisted by watsonx Code Assistant \n{\n  \'pom.xml\': \'<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">\\n  <modelVersion>4.0.0</modelVersion>\\n  <groupId>com.example</groupId>\\n  <artifactId>demo</artifactId>\\n  <version>0.0.1-SNAPSHOT</version>\\n  <dependencies>\\n    <dependency>\\n      <groupId>org.springframework.boot</groupId>\\n      <artifactId>spring-boot-starter-web</artifactId>\\n    </dependency>\\n  </dependencies>\\n  <build>\\n    <plugins>\\n      <plugin>\\n        <groupId>org.springframework.boot</groupId>\\n        <artifactId>spring-boot-maven-plugin</artifactId>\\n      </plugin>\\n    </plugins>\\n  </build>\\n</project>\',\n  \'src/main

### 5. 생성된 스프링부트 코드를 **java-todo-app** 기본 디렉토리에 저장

In [13]:
# Generate java files from LLM output

def generateAppFiles(response_str):
    # Extract dictionary part from string
    start_idx = response_str.find("{")
    end_idx = response_str.rfind("}") + 1
    code_dict_str = response_str[start_idx:end_idx]

    # Convert string to dictionary
    code_dict = ast.literal_eval(code_dict_str)

    # Define base directory
    base_dir = "./java-todo-app"

    # Create directory structure and save files
    for filepath, content in code_dict.items():
        full_path = os.path.join(base_dir, filepath)
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        
        with open(full_path, "w", encoding="utf-8") as file:
            file.write(content)

    print(f"Project structure created under '{base_dir}'")

    return code_dict_str

code_dict_str = generateAppFiles(response)
code_dict_str

Project structure created under './java-todo-app'


'{\n  \'pom.xml\': \'<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">\\n  <modelVersion>4.0.0</modelVersion>\\n  <groupId>com.example</groupId>\\n  <artifactId>demo</artifactId>\\n  <version>0.0.1-SNAPSHOT</version>\\n  <dependencies>\\n    <dependency>\\n      <groupId>org.springframework.boot</groupId>\\n      <artifactId>spring-boot-starter-web</artifactId>\\n    </dependency>\\n  </dependencies>\\n  <build>\\n    <plugins>\\n      <plugin>\\n        <groupId>org.springframework.boot</groupId>\\n        <artifactId>spring-boot-maven-plugin</artifactId>\\n      </plugin>\\n    </plugins>\\n  </build>\\n</project>\',\n  \'src/main/java/com/example/DemoApplication.java\': \'package com.example;\\n\\nimport org.springframework.boot.SpringApplication;\\nimport org.springframework.boot.autoconfigure.SpringBootApplication;\\n\\n@Spring

### 6. mvn을 사용하여 생성된 **java-todo-app** 컴파일 및 실행
새로운 터미널을 열어 아래 커맨드를 실행해 주세요.
* cd <path/to/dir>/java-todo-app
* mvn clean
* mvn spring-boot:run


### 7. WCA API를 사용하여 잠재적 오류 수정

마이그레이션된 자바 애플리케이션을 컴파일/실행할 때 오류가 발생하는 경우, 다음 단계에 따라 WCA API를 사용하여 해결할 수 있습니다:
1. 새로운 **error.txt** 파일을 생성합니다. 해당 오류를 **error.txt**에 복사합니다. 
-   (*제안: 여러 오류가 있는 경우, 처음 몇 개의 오류만 복사하세요*)
- **mvn clean** 실행 시 오류는 다음과 같을 수 있으며, 여기서는 처음 두 개의 오류를 선택했습니다:

    - <img src="https://github.ibm.com/watsonx-apac/wca-workshop-2025/tree/main/6-wca-api-python-to-java-modernization/images/mvn_clean_error.png" width="400">
    - <img src="./images/mvn_clean_error.png" width="400">

2. **error.txt**를 **java-todo-app** 디렉토리 내에 저장하고 아래 코드 블록으로 진행하세요.

In [14]:
def read_error_log(error_file="./java-todo-app/error.txt", cleaned_error_file="./java-todo-app/error_clean.txt", max_lines=30):
    
    """Reads the error log from error.txt, removes unnecessary text before 'java-todo-app/src', 
       saves the cleaned error log as error_clean.txt, and returns only the first N lines (max lines).
    """
    
    if not os.path.exists(error_file):
        print("No error log found.")
        return "No error log found."

    else:
        cleaned_lines = []
        
        with open(error_file, "r", encoding="utf-8") as f:
            for line in f:
                # If "java-todo-app/src" is in the line, remove everything before it
                if "/java-todo-app/" in line:
                    line = re.sub(r".*?(java-todo-app/)", r"/\1", line)
                
                cleaned_lines.append(line.strip())

        # Save the full cleaned error log as error_clean.txt
        with open(cleaned_error_file, "w", encoding="utf-8") as f:
            f.write("\n".join(cleaned_lines))

        print(f"\n✅ Read and cleaned error.txt! Saved as {cleaned_error_file}.\n")
        
        # Return only the first `max_lines` of cleaned lines
        return "\n".join(cleaned_lines[:max_lines])
    
error_str = read_error_log()


✅ Read and cleaned error.txt! Saved as ./java-todo-app/error_clean.txt.



In [15]:
def fixCodeErrors(code_dict_str, error_str):
    
    
    prompt = f""" Here's my springboot application present in dict format, where keys are filepaths and corresponding values contain code.
                  Springboot applcation: \n ``` {code_dict_str} ```

                  Upon running the app I'm getting the following error: \n ``` {error_str} ```

                  Your task is to: 
                  1. Analyze the error 
                  2. Update the code to resolve the error to run the application successfully.
                  
            """
    
    response = call_wca_api(prompt)

    return response

respose_errorlog = fixCodeErrors(code_dict_str, error_str)
respose_errorlog

WCA API response: 
  Sure, I'd be happy to help you with that. Here's what I can do:

1. **Analyze the error:**

The error message suggests that the `spring-boot-maven-plugin` is not configured correctly in the `pom.xml` file. And the `spring-boot-starter-web` dependency is also missing a version number.

2. **Update the code to resolve the error:**

To resolve the error, you can update the `pom.xml` file to include the `spring-boot-maven-plugin` and specify the version of the `spring-boot-starter-web` dependency. Here's the updated `pom.xml` file:

```xml
// Assisted by watsonx Code Assistant 
// watsonx Code Assistant did not check whether this code suggestion might be similar to third party code.
<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>
  <groupId>com.example</groupId>
  <artifactId>d

'Sure, I\'d be happy to help you with that. Here\'s what I can do:\n\n1. **Analyze the error:**\n\nThe error message suggests that the `spring-boot-maven-plugin` is not configured correctly in the `pom.xml` file. And the `spring-boot-starter-web` dependency is also missing a version number.\n\n2. **Update the code to resolve the error:**\n\nTo resolve the error, you can update the `pom.xml` file to include the `spring-boot-maven-plugin` and specify the version of the `spring-boot-starter-web` dependency. Here\'s the updated `pom.xml` file:\n\n```xml\n// Assisted by watsonx Code Assistant \n// watsonx Code Assistant did not check whether this code suggestion might be similar to third party code.\n<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">\n  <modelVersion>4.0.0</modelVersion>\n  <groupId>com.example</groupId>\n  <artifactId>de

- 다음으로, WCA API 코드 수정 제안에 따라 해당 자바 파일을 업데이트하세요.
- 섹션 6에서 설명한 대로 애플리케이션을 다시 컴파일하고 실행하세요.
    - mvn clean
    - mvn spring-boot:run
- 브라우저에서 다음 URL을 방문하여 스프링부트 애플리케이션을 사용하세요: **http://127.0.0.1:8080/**
- 포트 정보는 다음 위치에서 확인할 수 있습니다: /java-todo-app/src/main/resources/application.properties

**참고**: 경우에 따라 문제가 해결될 때까지 섹션 6과 7을 한 번 이상 반복해야 할 수 있습니다

#### 8. 성공적인 컴파일 및 실행

- **mvn clean**은 다음과 같이 보일 것입니다:
    - <img src="https://github.ibm.com/watsonx-apac/wca-workshop-2025/tree/main/6-wca-api-python-to-java-modernization/images/mvn_clean_success.png" width="400">
    - <img src="./images/mvn_clean_success.png" width="400">
    

- **mvn spring-boot:run**은 다음과 같이 보일 것입니다:
    - <img src="https://github.ibm.com/watsonx-apac/wca-workshop-2025/tree/main/6-wca-api-python-to-java-modernization/images/mvn_run_p1.png" width="400">
    - <img src="./images/mvn_run_p1.png" width="400">

    - <img src="https://github.ibm.com/watsonx-apac/wca-workshop-2025/tree/main/6-wca-api-python-to-java-modernization/images/mvn_run_p2.png" width="400">
    - <img src="./images/mvn_run_p2.png" width="400">

- 마이그레이션된 스프링부트 기반 앱은 다음과 같이 보일 것입니다:
    - 새 작업을 추가할 수 있으며, 이는 **./java-todo-app/tasks.json**에 있는 **tasks.json**에 저장됩니다

    - <img src="https://github.ibm.com/watsonx-apac/wca-workshop-2025/tree/main/6-wca-api-python-to-java-modernization/images/springboot-app.png" width="400">
    - <img src="./images/springboot-app.png" width="400">